In [ ]:
%matplotlib inline
%load_ext autoreload
% autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import seaborn

import metropolis_ising
import exact_ising_model as exact

ising = metropolis_ising.MetropolisIsing(lattice_size=4, bond_energy=1, temperature=8, initial_temperature="hi", sweeps=70536)
ising.show_lattice()

In [ ]:
ising.metropolis()

ising.plot_magnetization()
ising.plot_energy()

In [ ]:
# Data gets normalized immediatly.
equilibrium = 5000
equilibrium_energy = ising.energy_history[equilibrium:] / ising.no_of_sites
equilibrium_magnetization = ising.magnet_history[equilibrium:] / ising.no_of_sites

In [ ]:
print(ising.binning_method(equilibrium_energy, 20, "Energy", show_plot=True))
print(ising.binning_method(np.absolute(np.absolute(equilibrium_magnetization)), 20, "Magnetization", show_plot=True))
print(ising.binning_method((equilibrium_magnetization*ising.no_of_sites)**2, 20, "Magnetization Squared", show_plot=True))

magnet_correlation_time, _ = ising.numpy_autocorrelation(np.absolute(equilibrium_magnetization))
print("Magnetization correlation time is {0} Monte Carlo Sweeps.".format(magnet_correlation_time))

energy_correlation_time, _ = ising.numpy_autocorrelation(equilibrium_energy)
print("Energy correlation time is {0} Monte Carlo Sweeps.".format(energy_correlation_time))

ising.sample_every_two_correlation_times(equilibrium_energy, equilibrium_magnetization, energy_correlation_time)

In [ ]:
def binning_temperature_range(lower, upper, lattice_size, step=0.2):
    energys = []
    energy_errors = []
    energy_correlations = []
    
    magnetizations = []
    magnetization_errors = []
    magnetization_correlations = []
    
    specific_heat = []
    specific_heat_errors = []
    specific_heat_correlations = []
    
    for temperature in np.arange(lower, upper, step):
        ising = metropolis_ising.MetropolisIsing(lattice_size=lattice, bond_energy=1, temperature=temperature, initial_temperature="hi", sweeps=10000)
        ising.metropolis()
        energy_correlation_time, energy, energy_error = ising.binning_method(ising.energy_history[5000:] / ising.no_of_sites, 10, "Energy")
        magnetization_correlation_time, magnetization, magnetization_error = ising.binning_method(np.absolute(ising.magnet_history[5000:] / ising.no_of_sites), 10, "Energy")
        
        c = ising.heat_capacity(ising.energy_history[5000:] / ising.no_of_sites, temperature)
        specific_heat.append((temperature, c))
        specific_heat_errors.append((temperature, ising.bootstrap_method(ising.energy_history[5000:] / ising.no_of_sites, 100, temperature, ising.heat_capacity)))
        
        energys.append((temperature, energy))
        energy_errors.append((temperature, energy_error))
        energy_correlations.append((temperature, energy_correlation_time))
        magnetizations.append((temperature, np.absolute(magnetization)))
        magnetization_errors.append((temperature, magnetization_error))
        magnetization_correlations.append((temperature, magnetization_correlation_time))
    
    return energys, energy_errors, energy_correlations, magnetizations, magnetization_errors, magnetization_correlations, specific_heat, specific_heat_errors, specific_heat_correlations

In [ ]:
lattice_size_range([4], 0.2, 5)

In [ ]:
def temperature_range(lower, upper, lattice_size, step=0.2):
    """Calculate energy and magnetization over a range of temperatures."""
    energy = []
    energy_error = []
    
    magnetization = []
    magnetization_error = []
    
    specific_heat = []
    specific_heat_error = []
    
    correlation_times = []
    
    for temperature in np.arange(lower, upper, step):
        ising = metropolis_ising.MetropolisIsing(lattice_size=lattice_size, bond_energy=1, temperature=temperature,
                                                 initial_temperature="lo", sweeps=10000)
        
        ising.metropolis()
        equilibrium = 2000

        energy_history = ising.energy_history[equilibrium:] / ising.no_of_sites
        magnet_history = ising.magnet_history[equilibrium:] / ising.no_of_sites
        correlation_time, acfunc = ising.numpy_autocorrelation(magnet_history)

        if np.isnan(correlation_time) or correlation_time <= 0:
            correlation_time = 1
        correlation_times.append((temperature, correlation_time))

        c = ising.heat_capacity(energy_history, temperature)
        specific_heat.append((temperature, c))
        specific_heat_error.append((temperature, ising.bootstrap_method(energy_history, 100, temperature, ising.heat_capacity)))
        
        magnet_samples = []
        energy_samples = []

        # Sample every two correlation times.
        for t in range(0, len(energy_history), 2 * int(np.ceil(correlation_time))):
            magnet_samples.append(magnet_history[t])
            energy_samples.append(energy_history[t])

        energy.append((temperature, np.mean(energy_samples)))
        energy_error.append((temperature, np.std(energy_samples) / np.sqrt(len(energy_samples))))
#         print((temperature, np.std(energy_samples)) / np.sqrt(len(energy_samples)))

        magnetization.append((temperature, np.mean(np.absolute(magnet_samples))))
        magnetization_error.append((temperature, np.std(np.absolute(magnet_samples)) / np.sqrt(len(np.absolute(magnet_samples)))))
    
    return energy, energy_error, magnetization, magnetization_error, specific_heat, specific_heat_error, correlation_times

In [ ]:
def lattice_size_range(lattice_sizes, lower, upper, step=0.2):
    lattice_sizes.sort()
    energy_data = []
    energy_error_data = []
    energy_correlation_data = []
    
    magnetization_data = []
    magnetization_error_data = []
    magnetization_correlation_data = []
    
    specific_heat_data = []
    specific_heat_error_data = []
    specific_heat_correlation_data = []
    for p in lattice_sizes:
        print("Lattice Size is {0}.".format(p))
        (energy_trange, 
         energy_error_trange, 
         energy_correlation_trange, 
         magnetization_trange, 
         magnetization_error_trange, 
         magnetization_correlation_trange, 
         specific_heat_trange, 
         specific_heat_error_trange, 
         specific_heat_correlation_trange) = binning_temperature_range(lower, upper, lattice=p, step=step)
        
        energy_data.append(energy_trange)
        energy_error_data.append(energy_error_trange)
        energy_correlation_data.append(energy_correlation_trange)

        magnetization_data.append(magnetization_trange)
        magnetization_error_data.append(magnetization_error_trange)
        magnetization_correlation_data.append(magnetization_correlation_trange)

        specific_heat_data.append(specific_heat_trange)
        specific_heat_error_data.append(specific_heat_error_trange)
        specific_heat_correlation_data.append(specific_heat_correlation_trange)
    
    #TODO: Add other correlation functions and different temperature range functions.
    for k in range(len(lattice_sizes)):
        show = False
        if k == range(len(lattice_sizes))[-1]:
            show = True
        ising.plot_correlation_time_range(magnetization_correlation_data[k], lattice_sizes[k], "Magnetization", show_plot=show)
    for k in range(len(lattice_sizes)):
        show = False
        exact_energy = None
        if k == range(len(lattice_sizes))[-1]:
            show = True
            exact_energy = exact.internal_energy(1, lower, upper)
        ising.plot_quantity_range(energy_data[k], energy_error_data[k], "Internal Energy per Spin", lattice_sizes[k], show_plot=show, exact=exact_energy)
    for k in range(len(lattice_sizes)):
        show = False
        exact_magnetization = None
        if k == range(len(lattice_sizes))[-1]:
            show = True
            exact_magnetization = exact.magnetization(1, lower, upper)
        ising.plot_quantity_range(magnetization_data[k], magnetization_error_data[k], "Magnetization per Spin", lattice_sizes[k], show_plot=show, exact=exact_magnetization)
    for k in range(len(lattice_sizes)):
        show = False
        exact_specific_heat = None
        if k == range(len(lattice_sizes))[-1]:
            show = True
            exact_specific_heat = exact.heat_capacity(1, lower, upper)
        ising.plot_quantity_range(specific_heat_data[k], specific_heat_error_data[k], "Specific Heat per Spin", lattice_sizes[k], show_plot=show, exact=exact_specific_heat)

In [ ]:
energy, energy_error, magnetization, magnetization_error, specific_heat, specific_heat_error, correlation_times = temperature_range(0.2, 5, 0.2, 4)

In [ ]:
ising.plot_correlation_time_range(correlation_times, 4, "Magnetization")

ising.plot_quantity_range(energy, energy_error, "Internal Energy per Spin", 4)

ising.plot_quantity_range(magnetization, magnetization_error, "Magnetization per Spin", 4, exact=exact.magnetization(0.2, 5))

ising.plot_quantity_range(specific_heat, specific_heat_error, "Specific Heat per Spin", 4, exact=exact.heat_capacity(0.2, 5))

In [ ]:
energy4, energy_error4, magnetization4, magnetization_error4, specific_heat4, specific_heat_error4, correlation_times4 = temperature_range(0.2, 5, 0.2, 4)
energy7, energy_error7, magnetization7, magnetization_error7, specific_heat7, specific_heat_error7, correlation_times7 = temperature_range(0.2, 5, 0.2, 7)
energy10, energy_error10, magnetization10, magnetization_error10, specific_heat10, specific_heat_error10, correlation_times10 = temperature_range(0.2, 5, 0.2, 10)

In [ ]:
ising.plot_correlation_time_range(correlation_times4, 4, "Magnetization", show_plot=False)
ising.plot_correlation_time_range(correlation_times7, 7, "Magnetization", show_plot=False)
ising.plot_correlation_time_range(correlation_times10, 10, "Magnetization", save=True)

ising.plot_quantity_range(energy4, energy_error4, "Internal Energy per Spin", 4, show_plot=False)
ising.plot_quantity_range(energy7, energy_error7, "Internal Energy per Spin", 7, show_plot=False)
ising.plot_quantity_range(energy10, energy_error10, "Internal Energy per Spin", 10, exact=exact.internal_energy(0.2, 5), save=True)

ising.plot_quantity_range(magnetization4, magnetization_error4, "Magnetization per Spin", 4, show_plot=False)
ising.plot_quantity_range(magnetization7, magnetization_error7, "Magnetization per Spin", 7, show_plot=False)
ising.plot_quantity_range(magnetization10, magnetization_error10, "Magnetization per Spin", 10, exact=exact.magnetization(0.2, 5), save=True)

ising.plot_quantity_range(specific_heat4, specific_heat_error4, "Specific Heat per Spin", 4, show_plot=False)
ising.plot_quantity_range(specific_heat7, specific_heat_error7, "Specific Heat per Spin", 7, show_plot=False)
ising.plot_quantity_range(specific_heat10, specific_heat_error10, "Specific Heat per Spin", 10, exact=exact.heat_capacity(0.2, 5), save=True)

In [ ]:
def jackknife(data, calculation):
    split_data = np.array_split(data, 8)
    print(split_data)
    for n in split_data:
        print(len(n))

jackknife(equilibrium_energy, 4)